In [35]:
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
## langsmith에서 디버깅을 위해 사용하는 코드
import os 

LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_PROJECT="LCEL"

In [37]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{num}의 10배는?")
prompt

PromptTemplate(input_variables=['num'], template='{num}의 10배는?')

In [38]:
from langchain_openai import ChatOpenAI

chain = prompt | ChatOpenAI(model='gpt-3.5-turbo')
chain


PromptTemplate(input_variables=['num'], template='{num}의 10배는?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1474fea10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1475c7fd0>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [39]:
response = chain.invoke({"num":5})
response

AIMessage(content='50', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [40]:
response.content

'50'

**데이터를 효과적으로 전달하는 방법**

    - RunnablePassthrough는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있다.
    주로 RunnableParallel과 함께 사용되어 맵에서 새 키에 데이터를 할당하는데 사용한다.
    - RunnablePassthrough()이 단독으로 호출되면 단순히 입력을 받아 그대로 전달한다.
    assign(RunnablePassthrough.assgin(..)) 과 함께 호출되면 입력을 받아 assign 함수에 전달된 추가 인수를 추가한다.


In [41]:
from langchain_core.runnables import RunnablePassthrough

In [42]:
# RunnablePassthrough

chain.invoke({"num":2})

AIMessage(content='20입니다.', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 15, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

RuunablePassthrough 사용

In [43]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI(model='gpt-3.5-turbo')

runnable_chain.invoke(10)

AIMessage(content='100', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

RunnablePassthrough.assgin() 사용

In [44]:
RunnablePassthrough().invoke({"num":1})

{'num': 1}

In [45]:
(RunnablePassthrough.assign(new_num=lambda x: x["num"]*3)).invoke({"num":1})

{'num': 1, 'new_num': 3}

**RunnableParallel**

In [46]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스 생성. 여러 Runnable 인스턴스를 병렬로 실행

runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달. 입력된 데이터를 그대로 통과시킴
    passed = RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용해 'mult' lambda function 할당. 해당 함수는 num 키에 해당하는 값을 3배로 증가시킴
    extra = RunnablePassthrough().assign(multi = lambda x: x["num"]*3),
    # 'modified' 키워드 인자로 lambda function 전달, 이 함수는 입력된 딕셔너리의 num키에 해당하는 값에 1을 더함
    modified = lambda x: x["num"] +1
) 

# runnable 인스턴스에 {'num':1} 딕셔너리를 입력으로 전달해 invoke 메소드를 호출

runnable.invoke({"num":1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'multi': 3}, 'modified': 2}

In [47]:
# Chain도 RunnableParallel

chain1 = (
    {"num" : RunnablePassthrough()}
    | PromptTemplate.from_template("{num}의 10배는? \답변(결과만) : ")
    | ChatOpenAI(model='gpt-3.5-turbo')
)

chain2 = (
    {"num" : RunnablePassthrough()}
    | PromptTemplate.from_template("{num} 의 1/10배는? \답변:(결과만) : ")
    | ChatOpenAI(model='gpt-3.5-turbo')
)

In [48]:
combined_chain = RunnableParallel(a=chain1, b=chain2)
combined_chain.invoke({"num":10})

{'a': AIMessage(content='100', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 32, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}),
 'b': AIMessage(content='1', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 35, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})}

**RunnableLambda**

    - RunnableLambda로 사용자 정의 함수 맵핑

In [49]:
from langchain_core.runnables import RunnableLambda

In [50]:
def extra(x):
    ext = x['extra']
    return int(ext["num"] * int(ext["multi"]))


In [51]:
(runnable | RunnableLambda(extra)).invoke({"num":3})

27